In [7]:
%matplotlib inline

In [2]:
import pandas as pd
import json
from scipy.sparse import lil_matrix 
import scipy.spatial.distance
from sklearn.metrics import DistanceMetric
from sklearn.metrics.pairwise import cosine_distances

# RANKING MOVIES MOST SIMILAR TO TARGET

In [3]:
actor_name_map = {}
movie_actor_map = {}


with open("/Users/bricetikum/Desktop/INST414WORK/imdb_movies_2000to2022.prolific.json.txt", "r") as in_file:
    for line in in_file:
        
        this_movie = json.loads(line)
            
        for actor_id,actor_name in this_movie['actors']:
            actor_name_map[actor_id] = actor_name
            
        movie_actor_map[this_movie["imdb_id"]] = ({
            "movie": this_movie["title"],
            "actors": {a[0] for a in this_movie['actors']},
            "genres": this_movie["genres"]
        })


In [49]:
df = pd.DataFrame(movie_actor_map.values(), index=movie_actor_map.keys())
df[df["movie"] == "Avatar"]
df

,movie,actors,genres
tt0035423,Kate & Leopold,"{nm0005227, nm0000630, nm0000212, nm0413168}","[Comedy, Fantasy, Romance]"
tt0088751,The Naked Monster,"{nm0828288, nm0933983, nm0864851, nm0329491}","[Comedy, Horror, Sci-Fi]"
tt0096056,Crime and Punishment,"{nm0000457, nm0452288, nm0000417, nm0000603}",[Drama]
tt0113092,For the Cause,"{nm0001002, nm0001299, nm0936365, nm0923529}","[Action, Adventure, Drama]"
tt0116391,Gang,"{nm0412917, nm0006763, nm0007113, nm0310173}","[Action, Crime, Drama]"
...,...,...,...
tt9906278,A Classic Tour of Scotland: Footloose Special,{nm1644256},[Documentary]
tt9906644,Manoharam,"{nm1428724, nm2068971, nm10375007, nm7243877}","[Comedy, Drama]"
tt9906844,Footloose in England's Lake District,{nm1644256},[Documentary]
tt9907032,Footloose in London II: Undiscovered and Unusual,{nm1644256},[Documentary]


In [27]:
movie = df[df['movie'] == 'Avatar']
movie

,movie,actors,genres
tt0499549,Avatar,"{nm0735442, nm0941777, nm0000244, nm0757855}","[Action, Adventure, Fantasy]"


In [20]:
movie2 = df[df['movie'] == 'Avengers: Endgame']
movie2

,movie,actors,genres
tt4154796,Avengers: Endgame,"{nm0000375, nm0749263, nm1165110, nm0262635}","[Action, Adventure, Drama]"


In [23]:
movie3 = df[df['movie'] == 'Jurassic World']
movie3

,movie,actors,genres
tt0369610,Jurassic World,"{nm0695435, nm0339460, nm1339223, nm0397171}","[Action, Adventure, Sci-Fi]"


In [24]:
movie4 = df[df['movie'] == 'The Lion King']
movie4

,movie,actors,genres
tt6105098,The Lion King,"{nm0736622, nm0461498, nm0252230, nm2255973}","[Adventure, Animation, Drama]"


In [29]:
target_movie_id = 'tt0499549' # Avatar
target_movie_id2 = 'tt4154796' # Avengers: Endgame
target_movie_id3 = 'tt0369610' # Jurassic World 
target_movie_id4 = 'tt6105098' # The Lion King

In [30]:
target_movie = movie_actor_map[target_movie_id]
target_movie

{'movie': 'Avatar',
 'actors': {'nm0000244', 'nm0735442', 'nm0757855', 'nm0941777'},
 'genres': ['Action', 'Adventure', 'Fantasy']}

In [31]:
target_movie2 = movie_actor_map[target_movie_id2]
target_movie2

{'movie': 'Avengers: Endgame',
 'actors': {'nm0000375', 'nm0262635', 'nm0749263', 'nm1165110'},
 'genres': ['Action', 'Adventure', 'Drama']}

In [32]:
target_movie3 = movie_actor_map[target_movie_id3]
target_movie3

{'movie': 'Jurassic World',
 'actors': {'nm0339460', 'nm0397171', 'nm0695435', 'nm1339223'},
 'genres': ['Action', 'Adventure', 'Sci-Fi']}

In [33]:
target_movie4 = movie_actor_map[target_movie_id4]
target_movie4

{'movie': 'The Lion King',
 'actors': {'nm0252230', 'nm0461498', 'nm0736622', 'nm2255973'},
 'genres': ['Adventure', 'Animation', 'Drama']}

In [47]:
distances = []

target_actors = target_movie4["actors"]
for movie in movie_actor_map.values():
    these_actors = movie["actors"]
    
    numer = len(target_actors.intersection(these_actors))
    denom = len(target_actors.union(these_actors))
    
    jaccard_sim = numer / denom
    
    distances.append({
        "movie": movie,
        "similarity": jaccard_sim
    })

In [48]:
for similar_movie in sorted(distances, key=lambda x: x["similarity"], reverse=True)[:10]:
    print(similar_movie["movie"]["movie"], similar_movie["similarity"])
    for actor in similar_movie["movie"]["actors"]:
        print("\t", actor_name_map[actor])


The Lion King 1.0
	 Seth Rogen
	 Beyoncé
	 Chiwetel Ejiofor
	 Donald Glover
Day Zero 0.25
	 Chiwetel Ejiofor
Z for Zachariah 0.16666666666666666
	 Margot Robbie
	 Chris Pine
	 Chiwetel Ejiofor
The Fighting Temptations 0.14285714285714285
	 Cuba Gooding Jr.
	 Mike Epps
	 Nigel Washington
	 Beyoncé
Children of Men 0.14285714285714285
	 Clive Owen
	 Julianne Moore
	 Michael Caine
	 Chiwetel Ejiofor
It Was an Accident 0.14285714285714285
	 Max Beesley
	 Nicola Stapleton
	 James Bolam
	 Chiwetel Ejiofor
Austin Powers in Goldmember 0.14285714285714285
	 Michael York
	 Seth Green
	 Mike Myers
	 Beyoncé
Dirty Pretty Things 0.14285714285714285
	 Kriss Dosanjh
	 Audrey Tautou
	 Sophie Okonedo
	 Chiwetel Ejiofor
3 Blind Mice 0.14285714285714285
	 Sara Stewart
	 Emilia Fox
	 Edward Furlong
	 Chiwetel Ejiofor
Melinda and Melinda 0.14285714285714285
	 Wallace Shawn
	 Will Ferrell
	 Vinessa Shaw
	 Chiwetel Ejiofor
